In [ ]:
import numpy as np
import cv2
import pandas as pd
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
from matplotlib import pyplot as plt
from sklearn import metrics
import glob

In [ ]:
def feature_extraction(img):
    df = pd.DataFrame()
    img2 = img.reshape(-1)
    df['Original Image'] = img2
    num = 1
    kernels = []


    #genearate gabor filters
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
                    if num in [20,28,13,2,9,10,19,14,16,27,26,25,17,18,1,15]:
                        num+=1
                        continue
                    gabor_label = 'Gabor' + str(num)
                    ksize=9
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                    kernels.append(kernel)
                    fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img
                    num += 1
                    
########################################
#Geerate OTHER FEATURES and add them to the data frame
#Feature 3 is canny edge
    edges = cv2.Canny(img, 100,200)   #Image, min and max values
    edges1 = edges.reshape(-1)
    df['Canny Edge'] = edges1 #Add column to original dataframe

#Feature 4 is Roberts edge
    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df['Roberts'] = edge_roberts1

#Feature 5 is Sobel
    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df['Sobel'] = edge_sobel1

#Feature 6 is Scharr
    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df['Scharr'] = edge_scharr1

    #Feature 7 is Prewitt
    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['Prewitt'] = edge_prewitt1

    #Feature 8 is Gaussian with sigma=3
    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df['Gaussian s3'] = gaussian_img1

    #Feature 9 is Gaussian with sigma=7
    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df['Gaussian s7'] = gaussian_img3

    #Feature 10 is Median with sigma=3
    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    df['Median s3'] = median_img1

    #Feature 11 is Otsu Thresholding
    ret,otsu_img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    otsu_img1 = otsu_img.reshape(-1)
    df['Otsu'] = otsu_img1  #Add column to original dataframe


    return df

In [ ]:
image_dataset = pd.DataFrame();
path = "images/train_images/*.jpg"
for file in glob.glob(path):
    img1= cv2.imread(file)
    img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    X = feature_extraction(img)
    image_dataset = pd.concat([image_dataset,X],ignore_index = True)
    

In [ ]:
mask_dataset = pd.DataFrame();
path = "images/train_masks/*.png"
for file in glob.glob(path):
    # print(file)
    temp = pd.DataFrame()
    img1= cv2.imread(file)
    img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img = np.where(img>0,255,img)

    X = img.reshape(-1)
    temp['Labels'] = X
    mask_dataset = pd.concat([mask_dataset,temp],ignore_index = True)

In [ ]:
dataset = pd.concat([image_dataset,mask_dataset],axis=1)

In [ ]:
Y = dataset["Labels"].values

#Define the independent variables
X = dataset.drop(labels = ["Labels"], axis=1) 

#Split data into train and test to verify accuracy after fitting the model. 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=20)

In [ ]:
model = RandomForestClassifier(n_estimators = 25, random_state = 42,verbose=10)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
path = "images/test_images/*.jpg"
for file in glob.glob(path):
    
    img1= cv2.imread(file)
    img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    name = file.split("\\") 
    print(name[1] + " extraction start")
    X = feature_extraction(img)
    print(name[1] + " predict start")
    result = loaded_model.predict(X)
    segmented = result.reshape((img.shape))
    namePng = name[1].split(".")
    # print(namePng[0]+".png")
    plt.imsave('images/Segmented/'+ namePng[0]+".png", segmented)